In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#modelamiento
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

In [2]:
#nativos
import os
import unicodedata

#terceros
import pandas as pd
import pymssql
import missingno as msno
from collections import Counter
import scorecardpy as sc

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

In [3]:
import numpy as np
np.__version__

'1.19.2'

#### ACÁ SOLO CAMBIAR EL EXCEL BASE

In [4]:
data = pd.read_excel('AVANCE_6_new_target.xlsx', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,PERIODO,COD_UNICO,NOMBRE_COMPLETO,FLG_VIGENTE,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,FLG_POSICION,...,_30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,_50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,_20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_int_COVID_ACOGIO_SUSPENSION_PERFECTA,_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT,_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT,COVID_FICHA_LLENADA
0,202007,0011874248,SOCIEDAD DE INDUSTRIAS OLEAGINOSAS,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0
1,202007,0011871910,PESQUERA MARINHA,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0
2,202007,0011861230,BISAGRAS PERUANAS,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,False,False,0
3,202007,0011858538,V& J INGENIERIA Y CONSTRUCCION,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0
4,202007,0011858467,M & H GROUP,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0


### ENCONTRANDO LOS MEJORES CORTES 

In [6]:
from sklearn.metrics import roc_auc_score, recall_score

In [7]:
[col for col in data.columns if 'target' in col.lower()]

['PRE_TARGET_yoy',
 'target_yoy_quartil_03',
 'target_yoy_quartil_04',
 'target_yoy_quartil_05',
 'target_yoy_quartil_06',
 'target_yoy_quartil_07',
 'target_feve_reducir_salir',
 'target_seguircovid_reducir_salir',
 'cuasi_target_preocupa',
 'target_no_normales_ibk_sbs',
 'target_8_dias_atraso',
 'target_15_dias_atraso',
 'target_rcc_8_dias_atraso',
 'target_SF_8_dias_atraso_o_mas']

In [8]:
banca_a_analizar = 'BEL ZONAL 1'
banca_a_analizar_2 = 'BEL ZONAL 3' # CAMBIAR POR LA ZONA A BUSCAR ALERTAS
banca_mediana = 'BEL ZONAL 2'
meses = [202007, 202008, 202009]
bancas = [banca_mediana]

In [9]:
col_target = 'cuasi_target_preocupa'

data_opt = data[
    (data[col_target].notnull()) & (data['PERIODO'].astype(int).isin(meses)) &  (data['ZONAL'].isin(bancas))
]
data_opt[col_target].value_counts()

0    2333
1     269
Name: cuasi_target_preocupa, dtype: int64

In [10]:
from scipy.optimize import differential_evolution

for col in var_num:
    optimization = differential_evolution(
        lambda c: -1*recall_score(data_opt[col_target], np.array([0 if _ < c[0] else 1 for _ in data_opt[col]])), 
        [(data_opt[col].min(), data_opt[col].max())]
    )
    
    print(col, optimization['x'], '|||', -1*optimization['fun'], "|||",optimization['success'], '//////////////// ',
          data_opt[
              (data_opt[col] >= optimization["x"][0]) & (data_opt[col].notnull())
          ].shape) 
    
    if 'VAR' in col.upper() or 'CEF' in col.upper():
        data['cut_opt_for_' + col_target + '_' + col] = data[col].apply(lambda _: 0 if _ < optimization['x'][0] else 1)

COVID_PORCENTAJE_REDUCCION_PERSONAL [1.16280454] ||| 0.40892193308550184 ||| True ////////////////  (386, 1107)
COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL [2.37533714] ||| 0.7174721189591078 ||| True ////////////////  (897, 1107)
COVID_P_5_VARIACION_PORCENTUAL_VENTAS_2020_2019 [-78.60765108] ||| 1.0 ||| True ////////////////  (2600, 1107)
TRX_TIPOS_CASH_IN_MENORES_DE_U3M [0.1764608] ||| 0.20817843866171004 ||| True ////////////////  (670, 1107)
RIESGO [374.41438157] ||| 0.9628252788104089 ||| True ////////////////  (2466, 1107)
TRX_TIPOS_CASH_IN_CAIDA_SEGUIDA_U3M [0.7547587] ||| 0.26394052044609667 ||| True ////////////////  (753, 1107)
TRX_TIPOS_CASH_IN_CAIDA_10PERCENT_MES_ANT [0.84030251] ||| 0.37174721189591076 ||| True ////////////////  (1071, 1107)
TRX_TIPOS_CASH_IN_CAIDA_20PERCENT_MES_ANT [0.70707705] ||| 0.3382899628252788 ||| True ////////////////  (990, 1107)
TRX_TIPOS_CASH_IN_CAIDA_30PERCENT_MES_ANT [0.03446919] ||| 0.30111524163568776 ||| True ////////////////  (915, 1107)
TR

In [11]:
for col in [_ for _ in data.columns if 'TIENE' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CUT' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MENOR' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MAYOR' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'SUBIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NUM' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'INT' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CAIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NRO' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'COVID' in _.upper()]:
    data[col] = data[col].fillna(0)

In [12]:
cols_combine = [col for col in data.columns if set(data[col].unique()) == {1, 0} or 'NRO' in col.upper()]
#print(list(cols_combine))
print(len(cols_combine))

825


In [13]:
del data_opt

In [16]:
cols_combine = \
    [col for col in cols_combine if 'SOBREGIRO' not in col.upper() \
                                 and 'SCOTIA' not in col.upper() \
                                 and 'BCP' not in col.upper() \
                                 and 'BBVA' not in col.upper() \
                                 and 'BIF' not in col.upper() \
                                 and '%' not in col.upper()
    ]
print(list(cols_combine))

['FLG_VIGENTE', 'FLG_SUSPENDIDO', 'FLG_VENCIDA', 'FLG_CANCELADO', 'FLG_MONOLINEA', 'FLG_SOLO_REACTIVA', 'FLG_POSICION', 'DESPRIORIZACION_ADM', 'NRO_VECES_REACTIVA_IBK', 'TIENE_REACTIVA_IBK', 'TIENE_REACTIVA1_REACTIVA2_IBK', 'NRO_VECES_REACTIVA_SF', 'TIENE_REACTIVA_SF', 'TIENE_REACTIVA1_REACTIVA2_SF', 'TIENE_REACTIVA_CON_OTRA_EF', 'TIENE_REACTIVA_1_2_CON_OTRAS_EF', 'TIENE_SOLO_1_CREDITO_REACTIVA_SF', 'TRX_CAIDA_SEGUIDA_U3M_CASH_IN', 'TRX_CAIDA_10PERCENT_MES_ANTERIOR', 'TRX_CAIDA_20PERCENT_MES_ANTERIOR', 'TRX_CAIDA_30PERCENT_MES_ANTERIOR', 'TRX_CAIDA_40PERCENT_MES_ANTERIOR', 'TRX_CAIDA_50PERCENT_MES_ANTERIOR', 'TRX_SUBIDA_10PERCENT_MES_ANTERIOR', 'TRX_SUBIDA_20PERCENT_MES_ANTERIOR', 'TRX_SUBIDA_30PERCENT_MES_ANTERIOR', 'TRX_SUBIDA_40PERCENT_MES_ANTERIOR', 'TRX_SUBIDA_50PERCENT_MES_ANTERIOR', 'TRX_SUBIDA_SEGUIDA_U3M_CASH_IN', 'TRX_MENOR_CASH_IN_DE_U3M', 'TRX_MAYOR_CASH_IN_DE_U3M', 'TRX_CAIDA_SEGUIDA_O_NO_CASH_IN_U3M', 'NO_TIENE_TRX_COBRANZAS__MES_ACTUAL', 'NO_TIENE_TRX_DEPOSITOS__MES_ACTU

In [17]:
print(len(cols_combine))

759


In [18]:
for col in ['cambio_seguir_covid', 'TIENE_DIAS_DE_ATRASO_IBK_ESTE_MES', 'TIENE_DEUDA_VIGENTE_IBK'] + [col for col in data.columns if 'target' in col.lower()]:
    if col in cols_combine:
        cols_combine.remove(col)

In [19]:
import itertools
combine = list(itertools.combinations(cols_combine, 3))
len(combine)

54513680

In [20]:
data['PERIODO'].value_counts()

202007    1924
202008    1882
202009    1832
Name: PERIODO, dtype: int64

#### SOLO TOMAR 202007 Y 202008 ORUQEU SON LOS MESES QUE HASTA EL MONENTO TIENEN TARGET COMPLETO 2 MESES ENADELANTE

In [23]:
data_bel_zon_1= data[
    (data['ZONAL'].isin(bancas)) & (data['PERIODO'].astype(int).isin(meses))
]

#### VALIDACION DE ESTAR TOMANDO LO CORRECTO , DEBE SOLO APRECER LOS PERIODOS DE COSECHA COMPLETA Y LA ZONA INDICADA

In [24]:
data_bel_zon_1['PERIODO'].value_counts()

202007    897
202008    871
202009    834
Name: PERIODO, dtype: int64

In [25]:
data_bel_zon_1['ZONAL'].value_counts()

BEL ZONAL 2    2602
Name: ZONAL, dtype: int64

## GO!!!!!!!!!!!

In [ ]:
data.to_excel("AVANCE_7.xlsx", index=False)

#### EN LOS PRINT VERASL LAS VARIABLES LA PROBABILIDA PROMEDIO Y LA CANTIDAD HALLADA PARA IRA ANALIZANDO

In [ ]:
import random as rn
listado = []  # acumula lo scaoss ya''q ya validóoooooooo
aucc = {}

In [31]:
#assert False

In [ ]:
#rn.shuffle(combine)
#random.shuffle(number_list)
data_play_1 = data_bel_zon_1[data_bel_zon_1['target_SF_8_dias_atraso_o_mas'].notnull()].copy()
data_play_2 = data_bel_zon_1[data_bel_zon_1['cuasi_target_preocupa'].notnull()].copy()
#data_play_3 = data_bel_zon_1[data_bel_zon_1['target_yoy_quartil_05'].notnull()].copy()

for i in range(1000000):
    x, y, z = rn.choice(combine)
   # x = 'NRO_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC'#'_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC'  
#for el in combine:
#    x, y = el
    col_evaluate = '1_and_1_{}___{}___{}'.format(x, y, z)
    if col_evaluate not in listado and 'accionista' not in col_evaluate.lower() and 'gerencia' not in col_evaluate.lower() :
        listado.append(col_evaluate)
        
        lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1 and _[2] >= 1
        data_bel_zon_1[col_evaluate] = data_bel_zon_1[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        data_play_1[col_evaluate] = data_play_1[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        data_play_2[col_evaluate] = data_play_2[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        #data_play_3[col_evaluate] = data_play_3[[x, y]].apply(lambda_basico, axis=1).astype(int)
        
        result_1 = recall_score(data_play_1[col_evaluate], data_play_1['target_SF_8_dias_atraso_o_mas'])
        result_2 = recall_score(data_play_2[col_evaluate], data_play_2['cuasi_target_preocupa'])
        #result_3 = recall_score(data_play_3[col_evaluate], data_play_3['target_yoy_quartil_05'])
        
        evaluacion = np.mean([result_1, result_2], axis=0)
        coincidencias = data_bel_zon_1[data_bel_zon_1[col_evaluate] == 1].shape[0] 
        
        if evaluacion > 3 and coincidencias >= 6:
            aucc[col_evaluate] = evaluacion
            print(x, y, z, evaluacion, '////', result_1, result_2,  ' ==== ', coincidencias)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................